# Das Training eines einfachen neuronalen Netzes

Das Training von neuronalen Netzen beruht im Wesentlichen auf dem Lösen eines Optimierungsproblems. Dabei sollen die Gewichte bzw. Parameter des Netzes so gewählten werden, dass der Fehler bzgl. der Trainingsdaten minimal wird. 

In diesem Notebook wird zunächst ein einzelnes Neuron implementiert. Dieses wird anschließend zu einem Netz erweitert. 

In [ ]:
# Pakete laden
import math
import numpy as np
from scipy.optimize import minimize
from scipy.special import expit

## Das Neuron

Ein Neuron gewichtet einen Eingabevektor $\mathbb{R}^n$ mit einem Gewichtsvektor aus $\mathbb{R}^{(n+1)}$. Die letzte Komponente ist eine additive Konstante, der sogenannte Bias. Anschließend wird die Sigmoid-Funktion komponentenweise angewendet. Die Funktion `neuron` illustriert dies. Sie wird in der Form jedoch bei der Implementierung eines ganzen Netzes nicht mehr gebraucht.

In [16]:
from scipy.special import expit
def sigmoid(x): return expit(x) # same as 1 / (1 + math.exp(-x)) but more stable for big numbers
def sigmoidVect(x): return np.array([sigmoid(xi) for xi in x])
def neuron(w, b,x):
    z = w[-1]+sum([w[i] * x[i] for i in range(len(x))])
    return sigmoid(z)
neuron([3,2],-2,[1,3])

0.999983298578152

Um zu beurteilen, wie nah/ähnlich zwei Vektoren einander sind, wird der quadrierte euklidische Abstand benötigt:

In [6]:
def distance(x,y): # Quadrat des Abstands zwischen zwei Vektoren
    return sum([(x[i]-y[i])**2 for i in range(len(x))])

## Neuronales Netz

Als Nächstes soll ein kleines neuronales Netz programmiert werden. Das folgende definiert zwei Gewichtsmatrizen und zwei Vektoren. All diese Parameter des Netzes werden im Lernprozess später bestimmt. Außerdem werden Funktion definiert, mit denen man solche Matrizen in einen einzigen großen Vektor umwandelt und umgekehrt. Das ist nötig, weil die verwendete Optimierungsroutine nur einen Vektor als Eingabe erwartet.
Alle Vektoren und Matrizen werden nicht als Listen behandelt, sondern als Objekte vom Datentyp np.array. Das hat den Vorteil, dass man den Operator @ für die Multiplikation einer Matrix mit einem Vektor verwenden kann.

In [7]:
W1=np.zeros((3, 4)) # Erzeugung von je zwei Matrizen und Vektoren, zunächst mit Nullen gefüllt
W2=np.zeros((2, 3))
B1=np.zeros((3))
B2=np.zeros((2))
def M2V(W1,W2,B1,B2): # verwandelt die Vektoren und Matrizen in einen einzigen großen Vektor
    return np.hstack((W1.flatten(), W2.flatten(), B1.flatten(), B2.flatten()))
def V2M(x): # Umkehroperation zu M2V
    lenW1=np.prod(np.shape(W1))
    lenW2=np.prod(np.shape(W2))
    lenB1=np.prod(np.shape(B1))
    lenB2=np.prod(np.shape(B2))
    w1 = x[:lenW1].reshape(np.shape(W1))
    w2 = x[lenW1:lenW1+lenW2].reshape(np.shape(W2))
    b1 = x[lenW1+lenW2:lenW1+lenW2+lenB1].reshape(np.shape(B1))
    b2 = x[lenW1+lenW2+lenB1:].reshape(np.shape(B2))
    return [w1,w2,b1,b2]
x0 = M2V(W1, W2, B1, B2) # Die Matrizen als ein großer Vektor
[V2M(x0)[0]==W1, V2M(x0)[1]==W2, V2M(x0)[2]==B1, V2M(x0)[3]==B2] # Rückkonvertierung liefert wieder die Matrizen

[array([[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]]),
 array([[ True,  True,  True],
        [ True,  True,  True]]),
 array([ True,  True,  True]),
 array([ True,  True])]

Das kleine Beispiel-Netz besteht aus zwei Lagen von Neuronen, die nacheinader angewendet werden. Das Netz ist eine Funktion $R^4 \rightarrow R^2$. Die Parameter des Netzes werden als optionale Variablen angegeben.

In [8]:
def net(X,w1=W1,w2=W2,b1=B1,b2=B2): return sigmoidVect(w2@sigmoidVect(w1@X+b1)+b2)
net(np.array([1,2,3,4]))

array([0.5, 0.5])

## Beispiel: Kantendetektor

Die Eingaben des Netzes sind Vektoren aus dem $R^4$, die die vier Pixel eines 2x2-Bildes darstellen sollen. Die Ausgabeneuronen sollen hohe Werte bei vertikalen beziehungsweise horizontalen Strukturen annehmen.

In [17]:
trainingdata= [
    [[1,0,1,0],[1,0]], [[0,1,0,1],[1,0]], 
    [[1,1,0,0],[0,1]], [[0,0,1,1],[0,1]], 
    [[1,1,1,1],[0.5,0.5]],[[0,0,0,0],[0.5,0.5]], 
    [[0,1,0,0],[0.5,0.5]], [[1,0,0,0],[0.5,0.5]],  
    [[0.9,0.1,1,0],[0.9,0.1]], [[0.1,1,0,0.9],[0.9,0.1]],  
    [[1,0,1,1],[0.5,0.5]], [[0,1,1,1],[0.5,0.5]],  
    [[1,1,0,1],[0.5,0.5]], [[1,1,1,0],[0.5,0.5]]  ]

In [10]:
# Das Folgende ist die zu minimierende Zielfunktion des Lernens: 
def F(x):
    [w1,w2,b1,b2]=V2M(x)
    return sum([distance(d[1],net(d[0],w1,w2,b1,b2)) for d in trainingdata])
opt=minimize(F,M2V(W1,W2,B1,B2),method='BFGS')
[W1opt,W2opt,B1opt,B2opt]=V2M(opt.x)
# das Folgende ist das trainierte Netz
def netopt(X): return net(X,W1opt,W2opt,B1opt,B2opt)

In [11]:
netopt(np.array([1,1,0.1,0]))

array([1.29004597e-174, 1.00000000e+000])

In [12]:
netopt(np.array([1,0,0.9,0]))

array([0.63049359, 0.37050827])

In [13]:
netopt(np.array([1,1,1,1]))

array([0.63049359, 0.37050827])

In [14]:
netopt(np.array([1,1,0.01,0]))

array([1.29004597e-174, 1.00000000e+000])

## Arbeitsaufträge:
1) Verändern Sie die Trainingsdaten so, dass das Netz lernt, diagonale Linien zu erkennen.
2) Verändern Sie die Netzstruktur, so dass es vier Ausgabe-Neuronen gibt, je eine soll signalisieren, dass die Struktur überwiegend horizontal, vertikal, diagonal fallend oder diagonal steigend ist. Erhöhen Sie dazu auch die Zahl der Neuronen in der mittleren Schicht auf 5. Stellen Sie passende Trainingsdaten dafür zusammen.
